# Dans ce notebook , nous allons finetuné le diarizateur de Nemo dans le but de le spécifié sur nos données Lecteur du coran avec le ton Bamoun et lecture du Bamoun
 .

# Nouvelle section : Quelques fonctions utiles

In [ ]:
#fonction python qui renomme des fichiers : on conserve ce qui est avant le - et on retire le reste y compris le - . Exemple : 01 - Nka' Yùbshe devient 01 suivie de l'extension du fichier .
import os

def rename_files_in_directory(directory):
    # Parcourir tous les fichiers dans le répertoire spécifié
    i=0
    for filename in os.listdir(directory):
        # Vérifier si le fichier contient un tiret
        i=i+1
        if '_' in filename:
            # Séparer le nom du fichier et l'extension
            name= os.path.splitext(filename)
            extension=".mp3"
            # Conserver ce qui est avant le tiret
            new_name = "seg_sh_"+str(i)+extension
            # Construire les chemins complets pour renommer
            old_file_path = os.path.join(directory, filename)
            new_file_path = os.path.join(directory, new_name)
            # Renommer le fichier
            os.rename(old_file_path, new_file_path)
            print(f'Renamed: "{filename}" to "{new_name}"')

# Exemple d'utilisation
rename_files_in_directory("C:/Users/dell/Downloads/112/sh")

In [ ]:
#creation des fichiers rttm  nous avons décider de retenir les audios qui en leurs sein n'ont qu'un seul lecteur . Cela voudrait tout simplement dire que nos fichiers rttm n'ont qu'un seul segment .
import os
from pydub import AudioSegment

def create_rttm_file(audio_filepath, speaker_name):

    # Récupérer le nom du fichier sans extension
    filename = os.path.splitext(os.path.basename(audio_filepath))[0]

    # Charger l'audio pour obtenir sa durée
    audio = AudioSegment.from_file(audio_filepath)
    duration = len(audio) / 1000.0  # Durée en secondes

    # Créer le contenu du fichier RTTM
    rttm_content = f"SPEAKER {filename} 1 0.0 {duration:.3f} <NA> <NA> {speaker_name} <NA> <NA>\n"

    # Chemin du fichier RTTM
    rttm_filepath = f"{filename}.rttm"

    # Écrire le contenu dans le fichier RTTM
    with open(rttm_filepath, 'w') as rttm_file:
        rttm_file.write(rttm_content)

    print(f"RTTM file created: {rttm_filepath}")

for filename in os.listdir("C:/Users/dell/Downloads/112/ar"):
    name,extension= os.path.splitext(filename)
    create_rttm_file("C:/Users/dell/Downloads/112/ar/"+filename,"Lecteur1CoranArabeTonBamoun")

In [ ]:
import os

def list_files_in_directory(directory, output_file, prefix):
    # Vérifier si le dossier existe
    if not os.path.exists(directory):
        print(f"Le dossier {directory} n'existe pas.")
        return

    # Ouvrir le fichier de sortie en mode écriture
    with open(output_file, 'w') as file:
        # Parcourir tous les fichiers dans le répertoire spécifié
        for filename in os.listdir(directory):
            # Vérifier si c'est un fichier
            if os.path.isfile(os.path.join(directory, filename)):
                # Écrire le nom du fichier avec le préfixe dans le fichier de sortie
                file.write(f"{prefix}{filename}\n")

    print(f"La liste des fichiers a été enregistrée dans {output_file}")

# Exemple d'utilisation
list_files_in_directory("D:/stage 5GI/données/coran/Coran arabe bamoun/code/rttmfiles", "rttmfiles/rttm_file_list.txt", "/rttmfiles/")

In [ ]:
import os
from pydub import AudioSegment

def train_manifest_json(directory1, prefix1, prefix2, output_file):
    # Vérifier si le dossier existe
    if not os.path.exists(directory1):
        print(f"Le dossier {directory1} n'existe pas.")
        return

    # Ouvrir le fichier de sortie en mode écriture
    with open(output_file, 'w') as file:
        # Parcourir tous les fichiers dans le répertoire spécifié
        for audiofilename in os.listdir(directory1):
            name, extension = os.path.splitext(audiofilename)

            # Vérifier si c'est un fichier et si c'est un fichier audio (par exemple, .mp3)
            if os.path.isfile(os.path.join(directory1, audiofilename)) and extension in ['.mp3', '.wav']:

                audio_file_path = prefix1 + audiofilename
                rttm_file_path = prefix2 + name + ".rttm"

                # Charger l'audio pour obtenir sa durée
                audio = AudioSegment.from_file(os.path.join(directory1, audiofilename))
                duration = len(audio) / 1000.0  # Durée en secondes

                # Créer le contenu du manifeste
                train_manifest_content = f'{{"audio_filepath": "{audio_file_path}", "offset": 0, "duration": {duration}, "label": "infer", "text": "-", "num_speakers": 1, "rttm_filepath": "{rttm_file_path}"}}\n'
                file.write(train_manifest_content)

    print(f"La liste des fichiers a été enregistrée dans {output_file}")

# Exemple d'utilisation
train_manifest_json(
    "D:/stage 5GI/données/coran/Coran arabe bamoun/finetuning modèle de diarization/audiofiles",
    "/audiofiles/",
    "/rttmfiles/",
    "D:/stage 5GI/données/coran/Coran arabe bamoun/finetuning modèle de diarization/train_manifest.json"
)

In [ ]:
!apt-get update && apt-get install -y libsndfile1 ffmpeg


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,244 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,718 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,915 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [55.7 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,2

In [ ]:
#Verification de la version 2.1.0rc0
import nemo
print(nemo.__version__)

In [ ]:
#verification 2.1.0rc0
!pip uninstall nemo-toolkit
!pip install nemo-toolkit==2.1.0rc0

Found existing installation: nemo-toolkit 2.4.0rc0
Uninstalling nemo-toolkit-2.4.0rc0:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/examples/*
    /usr/local/lib/python3.11/dist-packages/nemo/*
    /usr/local/lib/python3.11/dist-packages/nemo_toolkit-2.4.0rc0.dist-info/*
    /usr/local/lib/python3.11/dist-packages/scripts/*
    /usr/local/lib/python3.11/dist-packages/tests/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.11/dist-packages/examples/basic_example.py
    /usr/local/lib/python3.11/dist-packages/examples/cudf-example.py
    /usr/local/lib/python3.11/dist-packages/examples/cupy-example.py
    /usr/local/lib/python3.11/dist-packages/examples/readme_example.py
    /usr/local/lib/python3.11/dist-packages/examples/ucx/client_initialize.py
    /usr/local/lib/python3.11/dist-packages/examples/ucx/local_cuda_cluster.py
    /usr/local/lib/python3.11/dist-packages/tests/conversion_test.py
    /usr/local/lib/python3.11/dist-packages/tests/databa

In [ ]:
!pip install git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[asr]

DEPRECATION: git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[asr] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Cloning https://github.com/NVIDIA/NeMo.git (to revision main) to /tmp/pip-install-calrl_c8/nemo-toolkit_8ffb09bb319f4a14b0ef976cf737744c
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/NeMo.git /tmp/pip-install-calrl_c8/nemo-toolkit_8ffb09bb319f4a14b0ef976cf737744c
  Resolved https://github.com/NVIDIA/NeMo.git to commit 28db90438ad46a1c9a463e33dc20554cc03f2663
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata 

In [ ]:
!pip install Cython packaging

In [ ]:
!git clone https://github.com/NVIDIA/NeMo.git

Cloning into 'NeMo'...
remote: Enumerating objects: 241436, done.
remote: Counting objects: 100% (1474/1474), done.
remote: Compressing objects: 100% (711/711), done.
remote: Total 241436 (delta 1140), reused 788 (delta 763), pack-reused 239962 (from 4)
Receiving objects: 100% (241436/241436), 452.44 MiB | 46.58 MiB/s, done.
Resolving deltas: 100% (181168/181168), done.


In [ ]:
from google.colab import drive

# 1. Monter Google Drive
drive.mount('/content/gdrive')
#drive.flush_and_unmount()

Mounted at /content/gdrive


In [ ]:
!python /content/NeMo/examples/speaker_tasks/diarization/neural_diarizer/sortformer_diar_train.py \
  --config-path='../conf/neural_diarizer' \
  --config-name='sortformer_diarizer_hybrid_loss_4spk-v1.yaml' \
  trainer.devices=1 \
  model.train_ds.manifest_filepath='/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/finetuning_modele_de_diarization/train_manifest.json' \
  model.validation_ds.manifest_filepath='/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/finetuning_modele_de_diarization/test_manifest.json' \
  exp_manager.name='sample_train' \
  exp_manager.exp_dir=./sortformer_diar_train \
  +model.pretrained_state_dict="/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/finetuning_modele_de_diarization/fixed_model.ckpt"
  #exp_manager.resume_if_exists=True \
  #exp_manager.resume_from_checkpoint="/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/finetuning_modele_de_diarization/fixed_model.ckpt"


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
        window_size: 0.025
        sample_rate: ${model.sample_rate}
        window_stride: 0.01
        window: hann
        features: 80
        n_fft: 512
        frame_splicing: 1
        dither: 1.0e-05
      sortformer_modules:
        _target_: nemo.collections.asr.modules.sortformer_modules.SortformerModules
        num_spks: ${model.max_num_of_spks}
        dropout_rate: 0.5
        fc_d_model: ${model.model_defaults.fc_d_model}
        tf_d_model: ${model.model_defaults.tf_d_model}
      encoder:
        _target_: nemo.collections.asr.modules.ConformerEncoder
        feat_in: ${model.preprocessor.features}
        feat_out: -1
        n_layers: 18
        d_model: ${model.model_defaults.fc_d_model}
        subsampling: dw_striding
        subsampling_factor: 8
        subsampling_conv_channels: 256
        causal_downsampling: false
        ff_expansion_factor: 4
        self_attention_model: rel_po

In [ ]:
import nemo.collections.asr as nemo_asr

try:
    model = nemo_asr.models.EncDecSpeakerLabelModel.restore_from(
        "/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/finetuning_modele_de_diarization/modèle finetuné/checkpoints/sample_train.nemo"
    )
    print("Modèle chargé avec succès pour l'inférence.")
except Exception as e:
    print(f"Erreur lors du chargement du modèle : {e}")

[NeMo W 2025-05-08 17:05:45 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/finetuning_modele_de_diarization/train_manifest.json
    sample_rate: 16000
    num_spks: 4
    session_len_sec: 90
    soft_label_thres: 0.5
    soft_targets: false
    labels: null
    batch_size: 8
    shuffle: true
    num_workers: 18
    validation_mode: false
    use_lhotse: false
    use_bucketing: true
    num_buckets: 10
    bucket_duration_bins:
    - 10
    - 20
    - 30
    - 40
    - 50
    - 60
    - 70
    - 80
    - 90
    pin_memory: true
    min_duration: 10
    max_duration: 90
    batch_duration: 400
    quadratic_duration: 1200
    bucket_buffer_size: 20000
    shuffle_buffer_size: 10000
    window_stride: 0.01
    subsampling_factor: 8
    
[NeMo 

[NeMo I 2025-05-08 17:05:45 nemo_logging:393] PADDING: 16
[NeMo I 2025-05-08 17:05:46 nemo_logging:393] Normalized weights for PIL 0.5 and ATS 0.5
[NeMo I 2025-05-08 17:05:47 nemo_logging:393] Model SortformerEncLabelModel was successfully restored from /content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/finetuning_modele_de_diarization/modèle finetuné/checkpoints/sample_train.nemo.
Modèle chargé avec succès pour l'inférence.


In [ ]:
import shutil
import os

# Chemin du dossier source (où les résultats de l'entraînement sont sauvegardés)
source_folder = "./sortformer_diar_train/sample_train"

# Chemin du dossier cible dans votre Google Drive (MODIFIEZ CECI)
target_folder = "/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/finetuning_modele_de_diarization"

try:
    # Vérifier si le dossier source existe
    if not os.path.exists(source_folder):
        print(f"Le dossier source '{source_folder}' n'existe pas.")
    else:
        # Vérifier si le dossier cible existe, sinon le créer
        if not os.path.exists(target_folder):
            os.makedirs(target_folder)
            print(f"Le dossier cible '{target_folder}' a été créé.")

        # Récupérer la liste de tous les éléments dans le dossier source
        all_items = os.listdir(source_folder)

        # Déplacer chaque élément du dossier source vers le dossier cible
        for item in all_items:
            source_path = os.path.join(source_folder, item)
            target_path = os.path.join(target_folder, item)
            shutil.move(source_path, target_path)
            print(f"'{item}' a été déplacé vers '{target_folder}'.")

        print("Tous les éléments ont été déplacés avec succès.")

except Exception as e:
    print(f"Une erreur s'est produite lors du déplacement des fichiers : {e}")

'run_7' a été déplacé vers '/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/finetuning_modele_de_diarization'.
'nemo_log_globalrank-0_localrank-0.txt' a été déplacé vers '/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/finetuning_modele_de_diarization'.
'version_0' a été déplacé vers '/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/finetuning_modele_de_diarization'.
'run_6' a été déplacé vers '/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/finetuning_modele_de_diarization'.
'run_5' a été déplacé vers '/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/finetuning_modele_de_diarization'.
'run_4' a été déplacé vers '/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/finetuning_modele_de_diarization'.
'version_1' a été déplacé vers '/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/finetuning_modele_de_diarization'.
'version_2' a été déplacé vers '/content/gdrive/MyDrive/UMMISCO_

#Code 1 à aller remplacer
/content/NeMo/examples/speaker_tasks/diarization/neural_diarizer/sortformer_diar_train.py

In [ ]:
# Copyright (c) 2024, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import lightning.pytorch as pl
from omegaconf import OmegaConf
from pytorch_lightning import seed_everything
import torch
from omegaconf.dictconfig import DictConfig
from omegaconf.base import ContainerMetadata
from typing import Any
torch.serialization.add_safe_globals([DictConfig, ContainerMetadata, Any, dict])

from nemo.collections.asr.models import SortformerEncLabelModel
from nemo.core.config import hydra_runner
from nemo.utils import logging
from nemo.utils.exp_manager import exp_manager

"""
Example training session (single node training)

python ./sortformer_diar_train.py --config-path='../conf/neural_diarizer' \
    --config-name='sortformer_diarizer_hybrid_loss_4spk-v1.yaml' \
    trainer.devices=1 \
    model.train_ds.manifest_filepath="<train_manifest_path>" \
    model.validation_ds.manifest_filepath="<dev_manifest_path>" \
    exp_manager.name='sample_train' \
    exp_manager.exp_dir='./sortformer_diar_train'
"""

seed_everything(42)


@hydra_runner(config_path="../conf/neural_diarizer", config_name="sortformer_diarizer_hybrid_loss_4spk-v1.yaml")
def main(cfg):
    """Main function for training the sortformer diarizer model."""
    logging.info(f'Hydra config: {OmegaConf.to_yaml(cfg)}')
    trainer = pl.Trainer(**cfg.trainer)
    exp_manager(trainer, cfg.get("exp_manager", None))

    # Instanciation du modèle
    sortformer_model = SortformerEncLabelModel(cfg=cfg.model, trainer=trainer)

    if cfg.model.get("pretrained_state_dict", None) is not None:
        import torch
        logging.info(f"Loading pretrained state dict from {cfg.model.pretrained_state_dict}")
        state_dict = torch.load(cfg.model.pretrained_state_dict, map_location="cuda",weights_only=False)
        sortformer_model.load_state_dict(state_dict, strict=False)
        logging.info("Les poids pré-entraînés ont été chargés manuellement.")
    else:
        sortformer_model.maybe_init_from_pretrained_checkpoint(cfg)

    trainer.fit(sortformer_model)

    if hasattr(cfg.model, 'test_ds') and cfg.model.test_ds.manifest_filepath is not None:
        if sortformer_model.prepare_test(trainer):
            trainer.test(sortformer_model)


if __name__ == '__main__':
    main()

#Code 2 à remplacer
dans /usr/local/lib/python3.11/dist-packages/nemo/collections/asr/data/audio_to_diar_label.py



In [ ]:
# Copyright (c) 2022, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
from collections import OrderedDict
from statistics import mode
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch

from nemo.collections.asr.parts.utils.offline_clustering import get_argmin_mat
from nemo.collections.asr.parts.utils.speaker_utils import convert_rttm_line, get_subsegments, prepare_split_data
from nemo.collections.common.parts.preprocessing.collections import (
    DiarizationSpeechLabel,
    EndtoEndDiarizationSpeechLabel,
)
from nemo.core.classes import Dataset
from nemo.core.neural_types import AudioSignal, EncodedRepresentation, LengthsType, NeuralType, ProbsType
from nemo.utils import logging


def get_scale_mapping_list(uniq_timestamps):
    """
    Call get_argmin_mat function to find the index of the non-base-scale segment that is closest to the
    given base-scale segment. For each scale and each segment, a base-scale segment is assigned.

    Args:
        uniq_timestamps: (dict)
            The dictionary containing embeddings, timestamps and multiscale weights.
            If uniq_timestamps contains only one scale, single scale diarization is performed.

    Returns:
        scale_mapping_argmat (torch.tensor):

            The element at the m-th row and the n-th column of the scale mapping matrix indicates the (m+1)-th scale
            segment index which has the closest center distance with (n+1)-th segment in the base scale.

            - Example:
                `scale_mapping_argmat[2][101] = 85`

            In the above example, the code snippet means that 86-th segment in the 3rd scale (python index is 2) is
            mapped to the 102-th segment in the base scale. Thus, the longer segments bound to have more repeating
            numbers since multiple base scale segments (since the base scale has the shortest length) fall into the
            range of the longer segments. At the same time, each row contains N numbers of indices where N is number
            of segments in the base-scale (i.e., the finest scale).
    """
    timestamps_in_scales = []
    for key, val in uniq_timestamps['scale_dict'].items():
        timestamps_in_scales.append(torch.tensor(val['time_stamps']))
    session_scale_mapping_list = get_argmin_mat(timestamps_in_scales)
    scale_mapping_argmat = [[] for _ in range(len(uniq_timestamps['scale_dict'].keys()))]
    for scale_idx in range(len(session_scale_mapping_list)):
        scale_mapping_argmat[scale_idx] = session_scale_mapping_list[scale_idx]
    scale_mapping_argmat = torch.stack(scale_mapping_argmat)
    return scale_mapping_argmat


def extract_seg_info_from_rttm(rttm_lines, mapping_dict=None, target_spks=None):
    """
    Get RTTM lines containing speaker labels, start time and end time. target_spks contains two targeted
    speaker indices for creating groundtruth label files. Only speakers in target_spks variable will be
    included in the output lists.

    Args:
        uniq_id (str):
            Unique file ID that refers to an input audio file and corresponding RTTM (Annotation) file.
        rttm_lines (list):
            List containing RTTM lines in str format.
        mapping_dict (dict):
            Mapping between the estimated speakers and the speakers in the ground-truth annotation.
            `mapping_dict` variable is only provided when the inference mode is running in sequence-eval mode.
            Sequence eval mode uses the mapping between the estimated speakers and the speakers
            in ground-truth annotation.
    Returns:
        rttm_tup (tuple):
            Tuple containing lists of start time, end time and speaker labels.

    """
    stt_list, end_list, speaker_list, pairwise_infer_spks = [], [], [], []
    if target_spks:
        inv_map = {v: k for k, v in mapping_dict.items()}
        for spk_idx in target_spks:
            spk_str = f'speaker_{spk_idx}'
            if spk_str in inv_map:
                pairwise_infer_spks.append(inv_map[spk_str])
    for rttm_line in rttm_lines:
        start, end, speaker = convert_rttm_line(rttm_line)
        if target_spks is None or speaker in pairwise_infer_spks:
            end_list.append(end)
            stt_list.append(start)
            speaker_list.append(speaker)
    rttm_tup = (stt_list, end_list, speaker_list)
    return rttm_tup


def assign_frame_level_spk_vector(rttm_timestamps, round_digits, frame_per_sec, target_spks, min_spks=2):
    """
    Create a multi-dimensional vector sequence containing speaker timestamp information in RTTM.
    The unit-length is the frame shift length of the acoustic feature. The feature-level annotations
    `fr_level_target` will later be converted to base-segment level diarization label.

    Args:
        rttm_timestamps (list):
            List containing start and end time for each speaker segment label.
            `stt_list`, `end_list` and `speaker_list` are contained.
        frame_per_sec (int):
            Number of feature frames per second. This quantity is determined by
            `window_stride` variable in preprocessing module.
        target_spks (tuple):
            Speaker indices that are generated from combinations.
            If there are only one or two speakers,
            only a single `target_spks` variable is generated.

    Returns:
        fr_level_target (torch.tensor):
            Tensor containing label for each feature level frame.
    """
    stt_list, end_list, speaker_list = rttm_timestamps
    if len(speaker_list) == 0:
        return None
    else:
        sorted_speakers = sorted(list(set(speaker_list)))
        total_fr_len = int(max(end_list) * (10**round_digits))
        spk_num = max(len(sorted_speakers), min_spks)
        speaker_mapping_dict = {rttm_key: x_int for x_int, rttm_key in enumerate(sorted_speakers)}
        fr_level_target = torch.zeros(total_fr_len, spk_num)

        # If RTTM is not provided, then there is no speaker mapping dict in target_spks.
        # Thus, return a zero-filled tensor as a placeholder.
        for count, (stt, end, spk_rttm_key) in enumerate(zip(stt_list, end_list, speaker_list)):
            stt, end = round(stt, round_digits), round(end, round_digits)
            spk = speaker_mapping_dict[spk_rttm_key]
            stt_fr, end_fr = int(round(stt, 2) * frame_per_sec), int(round(end, round_digits) * frame_per_sec)
            fr_level_target[stt_fr:end_fr, spk] = 1
        return fr_level_target


import torch
import numpy as np
from typing import List, Tuple

def get_subsegments_to_timestamps(
    subsegments: List[Tuple[float, float]],
    feat_per_sec: int = 100,
    max_end_ts: float = None,
    decimals: int = 2
):
    """
    Convertit les timestamps (start, duration) de sous-segments en frames,
    puis calcule l'heure de fin (start + duration) pour chaque segment.

    Args:
        subsegments (List[Tuple[float, float]]):
            Une liste de tuples où chaque tuple est (start, duration). Par exemple :
            [[0.0, 0.16], [0.08, 0.16], ...]
            Peut aussi être un tuple unique.
        feat_per_sec (int, optional):
            Le nombre de frames par seconde. Par défaut 100.
        max_end_ts (float, optional):
            Si fourni, on limite (clippe) la valeur du timestamp final.
        decimals (int, optional):
            Le nombre de décimales pour l'arrondi. Par défaut 2.

    Returns:
        torch.Tensor: Un tenseur de forme [N, 2] contenant pour chaque segment [start_frame, end_frame].
    """
    # 1. Normaliser l'entrée : si on reçoit un unique tuple, on l'enveloppe dans une liste.
    if isinstance(subsegments, tuple):
        subsegments = [subsegments]

    # 2. Conversion en tenseur multiplié par feat_per_sec.
    seg_ts = torch.tensor(subsegments, dtype=torch.float32) * feat_per_sec

    # 3. Forcer la forme 2D :
    if seg_ts.ndim == 1:
        # Si le tenseur a 1D, on s'assure qu'il a un nombre pair d'éléments
        if seg_ts.numel() % 2 == 0:
            seg_ts = seg_ts.view(-1, 2)
        else:
            raise ValueError("Le nombre d’éléments dans le tenseur est inattendu (non pair).")
    else:
        # Dans le cas 2D, vérifier que la deuxième dimension vaut 2
        if seg_ts.size(1) != 2:
            # On essaie alors de le reformer si le nombre total d’éléments est pair
            if seg_ts.numel() % 2 == 0:
                seg_ts = seg_ts.view(-1, 2)
            else:
                raise ValueError(f"Impossible de reformer un tenseur de forme (-1,2) à partir d'un tenseur de forme {seg_ts.shape}.")

    # 4. Arrondi et conversion en entier
    ts = torch.round(seg_ts, decimals=decimals).long()

    # 5. Calculer la timestamp de fin sans assignation in-place : on commence par extraire start et duration
    start = ts[:, 0]
    duration = ts[:, 1]
    end = start + duration
    # On recompose le tenseur de sortie via torch.stack pour forcer un tenseur de forme [N, 2]
    ts_out = torch.stack([start, end], dim=1)

    # 6. Éventuellement, appliquer un clipping si max_end_ts est défini
    if max_end_ts is not None:
        ts_out = torch.clamp(ts_out, max=int(max_end_ts * feat_per_sec))

    return ts_out



def extract_frame_info_from_rttm(offset, duration, rttm_lines, round_digits=3):
    """
    Extracts RTTM lines containing speaker labels, start time, and end time for a given audio segment.

    Args:
        uniq_id (str): Unique identifier for the audio file and corresponding RTTM file.
        offset (float): The starting time offset for the segment of interest.
        duration (float): The duration of the segment of interest.
        rttm_lines (list): List of RTTM lines in string format.
        round_digits (int, optional): Number of decimal places to round the start and end times. Defaults to 3.

    Returns:
        rttm_mat (tuple): A tuple containing lists of start times, end times, and speaker labels.
        sess_to_global_spkids (dict): A mapping from session-specific speaker indices to global speaker identifiers.
    """
    rttm_stt, rttm_end = offset, offset + duration
    stt_list, end_list, speaker_list, speaker_set = [], [], [], []
    sess_to_global_spkids = dict()

    for rttm_line in rttm_lines:
        start, end, speaker = convert_rttm_line(rttm_line)

        # Skip invalid RTTM lines where the start time is greater than the end time.
        if start > end:
            continue

        # Check if the RTTM segment overlaps with the specified segment of interest.
        if (end > rttm_stt and start < rttm_end) or (start < rttm_end and end > rttm_stt):
            # Adjust the start and end times to fit within the segment of interest.
            start, end = max(start, rttm_stt), min(end, rttm_end)
        else:
            continue

        # Round the start and end times to the specified number of decimal places.
        end_list.append(round(end, round_digits))
        stt_list.append(round(start, round_digits))

        # Assign a unique index to each speaker and maintain a mapping.
        if speaker not in speaker_set:
            speaker_set.append(speaker)
        speaker_list.append(speaker_set.index(speaker))
        sess_to_global_spkids.update({speaker_set.index(speaker): speaker})

    rttm_mat = (stt_list, end_list, speaker_list)
    return rttm_mat, sess_to_global_spkids


def get_frame_targets_from_rttm(
    rttm_timestamps: list,
    offset: float,
    duration: float,
    round_digits: int,
    feat_per_sec: int,
    max_spks: int,
):
    """
    Create a multi-dimensional vector sequence containing speaker timestamp information in RTTM.
    The unit-length is the frame shift length of the acoustic feature. The feature-level annotations
    `feat_level_target` will later be converted to base-segment level diarization label.

    Args:
        rttm_timestamps (list):
            List containing start and end time for each speaker segment label.
            stt_list, end_list and speaker_list are contained.
        feat_per_sec (int):
            Number of feature frames per second.
            This quantity is determined by window_stride variable in preprocessing module.
        target_spks (tuple):
            Speaker indices that are generated from combinations. If there are only one or two speakers,
            only a single target_spks variable is generated.

    Returns:
        feat_level_target (torch.tensor):
            Tensor containing label for each feature level frame.
    """
    stt_list, end_list, speaker_list = rttm_timestamps
    sorted_speakers = sorted(list(set(speaker_list)))
    total_fr_len = int(duration * feat_per_sec)
    if len(sorted_speakers) > max_spks:
        logging.warning(
            f"Number of speakers in RTTM file {len(sorted_speakers)} exceeds the maximum number of speakers: "
            f"{max_spks}! Only {max_spks} first speakers remain, and this will affect frame metrics!"
        )
    feat_level_target = torch.zeros(total_fr_len, max_spks)
    for count, (stt, end, spk_rttm_key) in enumerate(zip(stt_list, end_list, speaker_list)):
        if end < offset or stt > offset + duration:
            continue
        stt, end = max(offset, stt), min(offset + duration, end)
        spk = spk_rttm_key
        if spk < max_spks:
            stt_fr, end_fr = int((stt - offset) * feat_per_sec), int((end - offset) * feat_per_sec)
            feat_level_target[stt_fr:end_fr, spk] = 1
    return feat_level_target


class _AudioMSDDTrainDataset(Dataset):
    """
    Dataset class that loads a json file containing paths to audio files,
    RTTM files and number of speakers. This Dataset class is designed for
    training or fine-tuning speaker embedding extractor and diarization decoder
    at the same time.

    Example:
    {"audio_filepath": "/path/to/audio_0.wav", "num_speakers": 2,
    "rttm_filepath": "/path/to/diar_label_0.rttm}
    ...
    {"audio_filepath": "/path/to/audio_n.wav", "num_speakers": 2,
    "rttm_filepath": "/path/to/diar_label_n.rttm}

    Args:
        manifest_filepath (str):
            Path to input manifest json files.
        multiscale_args_dict (dict):
            Dictionary containing the parameters for multiscale segmentation and clustering.
        emb_dir (str):
            Path to a temporary folder where segmentation information for embedding extraction is saved.
        soft_label_thres (float):
            Threshold that determines the label of each segment based on RTTM file information.
        featurizer:
            Featurizer instance for generating features from the raw waveform.
        window_stride (float):
            Window stride for acoustic feature. This value is used for calculating the numbers of feature-level frames.
        emb_batch_size (int):
            Number of embedding vectors that are trained with attached computational graphs.
        pairwise_infer (bool):
            This variable should be True if dataloader is created for an inference task.
        random_flip (bool):
            If True, the two labels and input signals are randomly flipped per every epoch while training.
    """

    @property
    def output_types(self) -> Optional[Dict[str, NeuralType]]:
        """Returns definitions of module output ports."""
        output_types = {
            "features": NeuralType(('B', 'T'), AudioSignal()),
            "feature_length": NeuralType(('B'), LengthsType()),
            "ms_seg_timestamps": NeuralType(('B', 'C', 'T', 'D'), LengthsType()),
            "ms_seg_counts": NeuralType(('B', 'C'), LengthsType()),
            "clus_label_index": NeuralType(('B', 'T'), LengthsType()),
            "scale_mapping": NeuralType(('B', 'C', 'T'), LengthsType()),
            "targets": NeuralType(('B', 'T', 'C'), ProbsType()),
        }

        return output_types

    def __init__(
        self,
        *,
        manifest_filepath: str,
        multiscale_args_dict: str,
        emb_dir: str,
        soft_label_thres: float,
        featurizer,
        window_stride,
        emb_batch_size,
        pairwise_infer: bool,
        random_flip: bool = True,
        global_rank: int = 0,
    ):
        super().__init__()
        self.collection = DiarizationSpeechLabel(
            manifests_files=manifest_filepath.split(','),
            emb_dict=None,
            clus_label_dict=None,
            pairwise_infer=pairwise_infer,
        )
        self.featurizer = featurizer
        self.multiscale_args_dict = multiscale_args_dict
        self.emb_dir = emb_dir
        self.round_digits = 2
        self.decim = 10**self.round_digits
        self.soft_label_thres = soft_label_thres
        self.pairwise_infer = pairwise_infer
        self.max_spks = 2
        self.frame_per_sec = int(1 / window_stride)
        self.emb_batch_size = emb_batch_size
        self.random_flip = random_flip
        self.global_rank = global_rank
        self.manifest_filepath = manifest_filepath
        self.multiscale_timestamp_dict = prepare_split_data(
            self.manifest_filepath,
            self.emb_dir,
            self.multiscale_args_dict,
            self.global_rank,
        )

    def __len__(self):
        return len(self.collection)

    def assign_labels_to_longer_segs(self, uniq_id, base_scale_clus_label):
        """
        Assign the generated speaker labels from the base scale (the finest scale) to the longer scales.
        This process is needed to get the cluster labels for each scale. The cluster labels are needed to
        calculate the cluster-average speaker embedding for each scale.

        Args:
            uniq_id (str):
                Unique sample ID for training.
            base_scale_clus_label (torch.tensor):
                Tensor variable containing the speaker labels for the base-scale segments.

        Returns:
            per_scale_clus_label (torch.tensor):
                Tensor variable containing the speaker labels for each segment in each scale.
                Note that the total length of the speaker label sequence differs over scale since
                each scale has a different number of segments for the same session.

            scale_mapping (torch.tensor):
                Matrix containing the segment indices of each scale. scale_mapping is necessary for reshaping the
                multiscale embeddings to form an input matrix for the MSDD model.
        """
        per_scale_clus_label = []
        self.scale_n = len(self.multiscale_timestamp_dict[uniq_id]['scale_dict'])
        uniq_scale_mapping = get_scale_mapping_list(self.multiscale_timestamp_dict[uniq_id])
        for scale_index in range(self.scale_n):
            new_clus_label = []
            scale_seq_len = len(self.multiscale_timestamp_dict[uniq_id]["scale_dict"][scale_index]["time_stamps"])
            for seg_idx in range(scale_seq_len):
                if seg_idx in uniq_scale_mapping[scale_index]:
                    seg_clus_label = mode(base_scale_clus_label[uniq_scale_mapping[scale_index] == seg_idx])
                else:
                    seg_clus_label = 0 if len(new_clus_label) == 0 else new_clus_label[-1]
                new_clus_label.append(seg_clus_label)
            per_scale_clus_label.extend(new_clus_label)
        per_scale_clus_label = torch.tensor(per_scale_clus_label)
        return per_scale_clus_label, uniq_scale_mapping

    def get_diar_target_labels(self, uniq_id, sample, fr_level_target):
        """
        Convert frame-level diarization target variable into segment-level target variable.
        Since the granularity is reduced from frame level (10ms) to segment level (100ms~500ms),
        we need a threshold value, `soft_label_thres`, which determines the label of each segment
        based on the overlap between a segment range (start and end time) and the frame-level target variable.

        Args:
            uniq_id (str):
                Unique file ID that refers to an input audio file and corresponding RTTM (Annotation) file.
            sample:
                `DiarizationSpeechLabel` instance containing sample information such as
                audio filepath and RTTM filepath.
            fr_level_target (torch.tensor):
                Tensor containing label for each feature-level frame.

        Returns:
            seg_target (torch.tensor):
                Tensor containing binary speaker labels for base-scale segments.
            base_clus_label (torch.tensor):
                Representative speaker label for each segment. This variable only has one speaker label
                for each base-scale segment.
                -1 means that there is no corresponding speaker in the target_spks tuple.
        """
        seg_target_list, base_clus_label = [], []
        self.scale_n = len(self.multiscale_timestamp_dict[uniq_id]['scale_dict'])
        subseg_time_stamp_list = self.multiscale_timestamp_dict[uniq_id]["scale_dict"][self.scale_n - 1]["time_stamps"]
        for seg_stt, seg_end in subseg_time_stamp_list:
            seg_stt_fr, seg_end_fr = int(seg_stt * self.frame_per_sec), int(seg_end * self.frame_per_sec)
            soft_label_vec_sess = torch.sum(fr_level_target[seg_stt_fr:seg_end_fr, :], axis=0) / (
                seg_end_fr - seg_stt_fr
            )
            label_int_sess = torch.argmax(soft_label_vec_sess)
            soft_label_vec = soft_label_vec_sess.unsqueeze(0)[:, sample.target_spks].squeeze()
            if label_int_sess in sample.target_spks and torch.sum(soft_label_vec_sess) > 0:
                label_int = sample.target_spks.index(label_int_sess)
            else:
                label_int = -1
            label_vec = (soft_label_vec > self.soft_label_thres).float()
            seg_target_list.append(label_vec.detach())
            base_clus_label.append(label_int)
        seg_target = torch.stack(seg_target_list)
        base_clus_label = torch.tensor(base_clus_label)
        return seg_target, base_clus_label

    def parse_rttm_for_ms_targets(self, sample):
        """
        Generate target tensor variable by extracting groundtruth diarization labels from an RTTM file.
        This function converts (start, end, speaker_id) format into base-scale (the finest scale) segment level
        diarization label in a matrix form.

        Example of seg_target:
            [[0., 1.], [0., 1.], [1., 1.], [1., 0.], [1., 0.], ..., [0., 1.]]

        Args:
            sample:
                `DiarizationSpeechLabel` instance containing sample information such as
                audio filepath and RTTM filepath.
            target_spks (tuple):
                Speaker indices that are generated from combinations. If there are only one or two speakers,
                only a single target_spks tuple is generated.

        Returns:
            clus_label_index (torch.tensor):
                Groundtruth clustering label (cluster index for each segment) from RTTM files for training purpose.
            seg_target  (torch.tensor):
                Tensor variable containing hard-labels of speaker activity in each base-scale segment.
            scale_mapping (torch.tensor):
                Matrix containing the segment indices of each scale. scale_mapping is necessary for reshaping the
                multiscale embeddings to form an input matrix for the MSDD model.

        """
        with open(sample.rttm_file, 'r') as file:
            rttm_lines = file.readlines()
        uniq_id = self.get_uniq_id_with_range(sample)
        rttm_timestamps = extract_seg_info_from_rttm(rttm_lines)
        fr_level_target = assign_frame_level_spk_vector(
            rttm_timestamps, self.round_digits, self.frame_per_sec, target_spks=sample.target_spks
        )
        seg_target, base_clus_label = self.get_diar_target_labels(uniq_id, sample, fr_level_target)
        clus_label_index, scale_mapping = self.assign_labels_to_longer_segs(uniq_id, base_clus_label)
        return clus_label_index, seg_target, scale_mapping

    def get_uniq_id_with_range(self, sample, deci=3):
        """
        Generate unique training sample ID from unique file ID, offset and duration. The start-end time added
        unique ID is required for identifying the sample since multiple short audio samples are generated from a single
        audio file. The start time and end time of the audio stream uses millisecond units if `deci=3`.

        Args:
            sample:
                `DiarizationSpeechLabel` instance from collections.

        Returns:
            uniq_id (str):
                Unique sample ID which includes start and end time of the audio stream.
                Example: abc1001_3122_6458

        """
        bare_uniq_id = os.path.splitext(os.path.basename(sample.rttm_file))[0]
        offset = str(int(round(sample.offset, deci) * pow(10, deci)))
        endtime = str(int(round(sample.offset + sample.duration, deci) * pow(10, deci)))
        uniq_id = f"{bare_uniq_id}_{offset}_{endtime}"
        return uniq_id

    def get_ms_seg_timestamps(self, sample):
        """
        Get start and end time of each diarization frame.

        Args:
            sample:
                `DiarizationSpeechLabel` instance from preprocessing.collections
        Returns:
            ms_seg_timestamps (torch.tensor):
                Tensor containing timestamps for each frame.
            ms_seg_counts (torch.tensor):
                Number of segments for each scale. This information is used for reshaping embedding batch
                during forward propagation.
        """
        uniq_id = self.get_uniq_id_with_range(sample)
        ms_seg_timestamps_list = []
        max_seq_len = len(self.multiscale_timestamp_dict[uniq_id]["scale_dict"][self.scale_n - 1]["time_stamps"])
        ms_seg_counts = [0 for _ in range(self.scale_n)]
        for scale_idx in range(self.scale_n):
            scale_ts_list = []
            for k, (seg_stt, seg_end) in enumerate(
                self.multiscale_timestamp_dict[uniq_id]["scale_dict"][scale_idx]["time_stamps"]
            ):
                stt, end = (
                    int((seg_stt - sample.offset) * self.frame_per_sec),
                    int((seg_end - sample.offset) * self.frame_per_sec),
                )
                scale_ts_list.append(torch.tensor([stt, end]).detach())
            ms_seg_counts[scale_idx] = len(
                self.multiscale_timestamp_dict[uniq_id]["scale_dict"][scale_idx]["time_stamps"]
            )
            scale_ts = torch.stack(scale_ts_list)
            scale_ts_padded = torch.cat([scale_ts, torch.zeros(max_seq_len - len(scale_ts_list), 2)], dim=0)
            ms_seg_timestamps_list.append(scale_ts_padded.detach())
        ms_seg_timestamps = torch.stack(ms_seg_timestamps_list)
        ms_seg_counts = torch.tensor(ms_seg_counts)
        return ms_seg_timestamps, ms_seg_counts

    def __getitem__(self, index):
        sample = self.collection[index]
        if sample.offset is None:
            sample.offset = 0
        clus_label_index, targets, scale_mapping = self.parse_rttm_for_ms_targets(sample)
        features = self.featurizer.process(sample.audio_file, offset=sample.offset, duration=sample.duration)
        feature_length = torch.tensor(features.shape[0]).long()
        ms_seg_timestamps, ms_seg_counts = self.get_ms_seg_timestamps(sample)
        if self.random_flip:
            torch.manual_seed(index)
            flip = torch.cat([torch.randperm(self.max_spks), torch.tensor(-1).unsqueeze(0)])
            clus_label_index, targets = flip[clus_label_index], targets[:, flip[: self.max_spks]]
        return features, feature_length, ms_seg_timestamps, ms_seg_counts, clus_label_index, scale_mapping, targets


class _AudioMSDDInferDataset(Dataset):
    """
    Dataset class that loads a json file containing paths to audio files,
    RTTM files and number of speakers. This Dataset class is built for diarization inference and
    evaluation. Speaker embedding sequences, segment timestamps, cluster-average speaker embeddings
    are loaded from memory and fed into the dataloader.

    Example:
    {"audio_filepath": "/path/to/audio_0.wav", "num_speakers": 2,
    "rttm_filepath": "/path/to/diar_label_0.rttm}
    ...
    {"audio_filepath": "/path/to/audio_n.wav", "num_speakers": 2,
    "rttm_filepath": "/path/to/diar_label_n.rttm}

    Args:
        manifest_filepath (str):
             Path to input manifest json files.
        emb_dict (dict):
            Dictionary containing cluster-average embeddings and speaker mapping information.
        emb_seq (dict):
            Dictionary containing multiscale speaker embedding sequence,
            scale mapping and corresponding segment timestamps.
        clus_label_dict (dict):
            Subsegment-level (from base-scale) speaker labels from clustering results.
        soft_label_thres (float):
            A threshold that determines the label of each segment based on RTTM file information.
        featurizer:
            Featurizer instance for generating features from raw waveform.
        seq_eval_mode (bool):
            If True, F1 score will be calculated for each speaker pair during inference mode.
        window_stride (float):
            Window stride for acoustic feature. This value is used for calculating the numbers of feature-level frames.
        use_single_scale_clus (bool):
            Use only one scale for clustering instead of using multiple scales of embeddings for clustering.
        pairwise_infer (bool):
            This variable should be True if dataloader is created for an inference task.
    """

    @property
    def output_types(self) -> Optional[Dict[str, NeuralType]]:
        """Returns definitions of module output ports."""
        output_types = OrderedDict(
            {
                "ms_emb_seq": NeuralType(('B', 'T', 'C', 'D'), SpectrogramType()),
                "length": NeuralType(tuple('B'), LengthsType()),
                "ms_avg_embs": NeuralType(('B', 'C', 'D', 'C'), EncodedRepresentation()),
                "targets": NeuralType(('B', 'T', 'C'), ProbsType()),
            }
        )
        return output_types

    def __init__(
        self,
        *,
        manifest_filepath: str,
        emb_dict: Dict,
        emb_seq: Dict,
        clus_label_dict: Dict,
        soft_label_thres: float,
        seq_eval_mode: bool,
        window_stride: float,
        use_single_scale_clus: bool,
        pairwise_infer: bool,
    ):
        super().__init__()
        self.collection = DiarizationSpeechLabel(
            manifests_files=manifest_filepath.split(','),
            emb_dict=emb_dict,
            clus_label_dict=clus_label_dict,
            seq_eval_mode=seq_eval_mode,
            pairwise_infer=pairwise_infer,
        )
        self.emb_dict = emb_dict
        self.emb_seq = emb_seq
        self.clus_label_dict = clus_label_dict
        self.round_digits = 2
        self.decim = 10**self.round_digits
        self.frame_per_sec = int(1 / window_stride)
        self.soft_label_thres = soft_label_thres
        self.pairwise_infer = pairwise_infer
        self.max_spks = 2
        self.use_single_scale_clus = use_single_scale_clus
        self.seq_eval_mode = seq_eval_mode

    def __len__(self):
        return len(self.collection)

    def parse_rttm_multiscale(self, sample):
        """
        Generate target tensor variable by extracting groundtruth diarization labels from an RTTM file.
        This function is only used when ``self.seq_eval_mode=True`` and RTTM files are provided. This function converts
        (start, end, speaker_id) format into base-scale (the finest scale) segment level diarization label in a matrix
        form to create target matrix.

        Args:
            sample:
                DiarizationSpeechLabel instance containing sample information such as audio filepath and RTTM filepath.
            target_spks (tuple):
                Two Indices of targeted speakers for evaluation.
                Example of target_spks: (2, 3)
        Returns:
            seg_target (torch.tensor):
                Tensor variable containing hard-labels of speaker activity in each base-scale segment.
        """
        if sample.rttm_file is None:
            raise ValueError(f"RTTM file is not provided for this sample {sample}")
        rttm_lines = open(sample.rttm_file).readlines()
        uniq_id = os.path.splitext(os.path.basename(sample.rttm_file))[0]
        mapping_dict = self.emb_dict[max(self.emb_dict.keys())][uniq_id]['mapping']
        rttm_timestamps = extract_seg_info_from_rttm(rttm_lines, mapping_dict, sample.target_spks)
        fr_level_target = assign_frame_level_spk_vector(
            rttm_timestamps, self.round_digits, self.frame_per_sec, sample.target_spks
        )
        seg_target = self.get_diar_target_labels_from_fr_target(uniq_id, fr_level_target)
        return seg_target

    def get_diar_target_labels_from_fr_target(self, uniq_id: str, fr_level_target: torch.Tensor) -> torch.Tensor:
        """
        Generate base-scale level binary diarization label from frame-level target matrix. For the given frame-level
        speaker target matrix fr_level_target, we count the number of frames that belong to each speaker and calculate
        ratios for each speaker into the `soft_label_vec` variable. Finally, `soft_label_vec` variable is compared
        with `soft_label_thres` to determine whether a label vector should contain 0 or 1 for each speaker bin.
        Note that seg_target variable has dimension of (number of base-scale segments x 2) dimension.

        Example of seg_target:
            [[0., 1.], [0., 1.], [1., 1.], [1., 0.], [1., 0.], ..., [0., 1.]]

        Args:
            uniq_id (str):
                Unique file ID that refers to an input audio file and corresponding RTTM (Annotation) file.
            fr_level_target (torch.tensor):
                frame-level binary speaker annotation (1: exist 0: non-exist) generated from RTTM file.

        Returns:
            seg_target (torch.tensor):
                Tensor variable containing binary hard-labels of speaker activity in each base-scale segment.

        """
        if fr_level_target is None:
            return None
        else:
            seg_target_list = []
            for seg_stt, seg_end, label_int in self.clus_label_dict[uniq_id]:
                seg_stt_fr, seg_end_fr = int(seg_stt * self.frame_per_sec), int(seg_end * self.frame_per_sec)
                soft_label_vec = torch.sum(fr_level_target[seg_stt_fr:seg_end_fr, :], axis=0) / (
                    seg_end_fr - seg_stt_fr
                )
                label_vec = (soft_label_vec > self.soft_label_thres).int()
                seg_target_list.append(label_vec)
            seg_target = torch.stack(seg_target_list)
            return seg_target

    def __getitem__(self, index):
        sample = self.collection[index]
        if sample.offset is None:
            sample.offset = 0

        uniq_id = os.path.splitext(os.path.basename(sample.audio_file))[0]
        scale_n = len(self.emb_dict.keys())
        _avg_embs = torch.stack([self.emb_dict[scale_index][uniq_id]['avg_embs'] for scale_index in range(scale_n)])

        if self.pairwise_infer:
            avg_embs = _avg_embs[:, :, self.collection[index].target_spks]
        else:
            avg_embs = _avg_embs

        if avg_embs.shape[2] > self.max_spks:
            raise ValueError(
                f" avg_embs.shape[2] {avg_embs.shape[2]} should be less than or equal to "
                f"self.max_num_speakers {self.max_spks}"
            )

        feats = []
        for scale_index in range(scale_n):
            repeat_mat = self.emb_seq["session_scale_mapping"][uniq_id][scale_index]
            feats.append(self.emb_seq[scale_index][uniq_id][repeat_mat, :])
        feats_out = torch.stack(feats).permute(1, 0, 2)
        feats_len = feats_out.shape[0]

        if self.seq_eval_mode:
            targets = self.parse_rttm_multiscale(sample)
        else:
            targets = torch.zeros(feats_len, 2).float()

        return feats_out, feats_len, targets, avg_embs


def _msdd_train_collate_fn(self, batch):
    """
    Collate batch of variables that are needed for raw waveform to diarization label training.
    The following variables are included in training/validation batch:

    Args:
        batch (tuple):
            Batch tuple containing the variables for the diarization training.
    Returns:
        features (torch.tensor):
            Raw waveform samples (time series) loaded from the audio_filepath in the input manifest file.
        feature lengths (time series sample length):
            A list of lengths of the raw waveform samples.
        ms_seg_timestamps (torch.tensor):
            Matrix containing the start time and end time (timestamps) for each segment and each scale.
            ms_seg_timestamps is needed for extracting acoustic features from raw waveforms.
        ms_seg_counts (torch.tensor):
            Matrix containing The number of segments for each scale. ms_seg_counts is necessary for reshaping
            the input matrix for the MSDD model.
        clus_label_index (torch.tensor):
            Groundtruth Clustering label (cluster index for each segment) from RTTM files for training purpose.
            clus_label_index is necessary for calculating cluster-average embedding.
        scale_mapping (torch.tensor):
            Matrix containing the segment indices of each scale. scale_mapping is necessary for reshaping the
            multiscale embeddings to form an input matrix for the MSDD model.
        targets (torch.tensor):
            Groundtruth Speaker label for the given input embedding sequence.
    """
    packed_batch = list(zip(*batch))
    features, feature_length, ms_seg_timestamps, ms_seg_counts, clus_label_index, scale_mapping, targets = packed_batch
    features_list, feature_length_list = [], []
    ms_seg_timestamps_list, ms_seg_counts_list, scale_clus_label_list, scale_mapping_list, targets_list = (
        [],
        [],
        [],
        [],
        [],
    )

    max_raw_feat_len = max([x.shape[0] for x in features])
    max_target_len = max([x.shape[0] for x in targets])
    max_total_seg_len = max([x.shape[0] for x in clus_label_index])

    for feat, feat_len, ms_seg_ts, ms_seg_ct, scale_clus, scl_map, tgt in batch:
        seq_len = tgt.shape[0]
        pad_feat = (0, max_raw_feat_len - feat_len)
        pad_tgt = (0, 0, 0, max_target_len - seq_len)
        pad_sm = (0, max_target_len - seq_len)
        pad_ts = (0, 0, 0, max_target_len - seq_len)
        pad_sc = (0, max_total_seg_len - scale_clus.shape[0])
        padded_feat = torch.nn.functional.pad(feat, pad_feat)
        padded_tgt = torch.nn.functional.pad(tgt, pad_tgt)
        padded_sm = torch.nn.functional.pad(scl_map, pad_sm)
        padded_ms_seg_ts = torch.nn.functional.pad(ms_seg_ts, pad_ts)
        padded_scale_clus = torch.nn.functional.pad(scale_clus, pad_sc)

        features_list.append(padded_feat)
        feature_length_list.append(feat_len.clone().detach())
        ms_seg_timestamps_list.append(padded_ms_seg_ts)
        ms_seg_counts_list.append(ms_seg_ct.clone().detach())
        scale_clus_label_list.append(padded_scale_clus)
        scale_mapping_list.append(padded_sm)
        targets_list.append(padded_tgt)

    features = torch.stack(features_list)
    feature_length = torch.stack(feature_length_list)
    ms_seg_timestamps = torch.stack(ms_seg_timestamps_list)
    clus_label_index = torch.stack(scale_clus_label_list)
    ms_seg_counts = torch.stack(ms_seg_counts_list)
    scale_mapping = torch.stack(scale_mapping_list)
    targets = torch.stack(targets_list)
    return features, feature_length, ms_seg_timestamps, ms_seg_counts, clus_label_index, scale_mapping, targets


def _msdd_infer_collate_fn(self, batch):
    """
    Collate batch of feats (speaker embeddings), feature lengths, target label sequences
    and cluster-average embeddings.

    Args:
        batch (tuple):
            Batch tuple containing feats, feats_len, targets and ms_avg_embs.
    Returns:
        feats (torch.tensor):
            Collated speaker embedding with unified length.
        feats_len (torch.tensor):
            The actual length of each embedding sequence without zero padding.
        targets (torch.tensor):
            Groundtruth Speaker label for the given input embedding sequence.
        ms_avg_embs (torch.tensor):
            Cluster-average speaker embedding vectors.
    """

    packed_batch = list(zip(*batch))
    feats, feats_len, targets, ms_avg_embs = packed_batch
    feats_list, flen_list, targets_list, ms_avg_embs_list = [], [], [], []
    max_audio_len = max(feats_len)
    max_target_len = max([x.shape[0] for x in targets])

    for feature, feat_len, target, ivector in batch:
        flen_list.append(feat_len)
        ms_avg_embs_list.append(ivector)
        if feat_len < max_audio_len:
            pad_a = (0, 0, 0, 0, 0, max_audio_len - feat_len)
            pad_t = (0, 0, 0, max_target_len - target.shape[0])
            padded_feature = torch.nn.functional.pad(feature, pad_a)
            padded_target = torch.nn.functional.pad(target, pad_t)
            feats_list.append(padded_feature)
            targets_list.append(padded_target)
        else:
            targets_list.append(target.clone().detach())
            feats_list.append(feature.clone().detach())

    feats = torch.stack(feats_list)
    feats_len = torch.tensor(flen_list)
    targets = torch.stack(targets_list)
    ms_avg_embs = torch.stack(ms_avg_embs_list)
    return feats, feats_len, targets, ms_avg_embs


class AudioToSpeechMSDDTrainDataset(_AudioMSDDTrainDataset):
    """
    Dataset class that loads a json file containing paths to audio files,
    rttm files and number of speakers. This Dataset class is designed for
    training or fine-tuning speaker embedding extractor and diarization decoder
    at the same time.

    Example:
    {"audio_filepath": "/path/to/audio_0.wav", "num_speakers": 2,
    "rttm_filepath": "/path/to/diar_label_0.rttm}
    ...
    {"audio_filepath": "/path/to/audio_n.wav", "num_speakers": 2,
    "rttm_filepath": "/path/to/diar_label_n.rttm}

    Args:
        manifest_filepath (str):
            Path to input manifest json files.
        multiscale_args_dict (dict):
            Dictionary containing the parameters for multiscale segmentation and clustering.
        emb_dir (str):
            Path to a temporary folder where segmentation information for embedding extraction is saved.
        soft_label_thres (float):
            A threshold that determines the label of each segment based on RTTM file information.
        featurizer:
            Featurizer instance for generating features from the raw waveform.
        window_stride (float):
            Window stride for acoustic feature. This value is used for calculating the numbers of feature-level frames.
        emb_batch_size (int):
            Number of embedding vectors that are trained with attached computational graphs.
        pairwise_infer (bool):
            This variable should be True if dataloader is created for an inference task.
    """

    def __init__(
        self,
        *,
        manifest_filepath: str,
        multiscale_args_dict: Dict,
        emb_dir: str,
        soft_label_thres: float,
        featurizer,
        window_stride,
        emb_batch_size,
        pairwise_infer: bool,
        global_rank: int,
    ):
        super().__init__(
            manifest_filepath=manifest_filepath,
            multiscale_args_dict=multiscale_args_dict,
            emb_dir=emb_dir,
            soft_label_thres=soft_label_thres,
            featurizer=featurizer,
            window_stride=window_stride,
            emb_batch_size=emb_batch_size,
            pairwise_infer=pairwise_infer,
            global_rank=global_rank,
        )

    def msdd_train_collate_fn(self, batch):
        """Collate batch of audio features, feature lengths, target label sequences for training."""
        return _msdd_train_collate_fn(self, batch)


class AudioToSpeechMSDDInferDataset(_AudioMSDDInferDataset):
    """
    Dataset class that loads a json file containing paths to audio files,
    rttm files and number of speakers. The created labels are used for diarization inference.

    Example:
    {"audio_filepath": "/path/to/audio_0.wav", "num_speakers": 2,
    "rttm_filepath": "/path/to/diar_label_0.rttm}
    ...
    {"audio_filepath": "/path/to/audio_n.wav", "num_speakers": 2,
    "rttm_filepath": "/path/to/diar_label_n.rttm}

    Args:
        manifest_filepath (str):
            Path to input manifest json files.
        emb_dict (dict):
            Dictionary containing cluster-average embeddings and speaker mapping information.
        emb_seq (dict):
            Dictionary containing multiscale speaker embedding sequence, scale mapping
            and corresponding segment timestamps.
        clus_label_dict (dict):
            Subsegment-level (from base-scale) speaker labels from clustering results.
        soft_label_thres (float):
            Threshold that determines speaker labels of segments depending on the overlap
            with groundtruth speaker timestamps.
        featurizer:
            Featurizer instance for generating features from raw waveform.
        use_single_scale_clus (bool):
            Use only one scale for clustering instead of using multiple scales of embeddings for clustering.
        seq_eval_mode (bool):
            If True, F1 score will be calculated for each speaker pair during inference mode.
        window_stride (float):
            Window stride for acoustic feature. This value is used for calculating the numbers of
            feature-level frames.
        pairwise_infer (bool):
            If True, this Dataset class operates in inference mode. In inference mode, a set of speakers
            in the input audio is split into multiple pairs of speakers and speaker tuples
            (e.g. 3 speakers: [(0,1), (1,2), (0,2)]) and then fed into the MSDD to merge the individual results.
    """

    def __init__(
        self,
        *,
        manifest_filepath: str,
        emb_dict: Dict,
        emb_seq: Dict,
        clus_label_dict: Dict,
        soft_label_thres: float,
        use_single_scale_clus: bool,
        seq_eval_mode: bool,
        window_stride: float,
        pairwise_infer: bool,
    ):
        super().__init__(
            manifest_filepath=manifest_filepath,
            emb_dict=emb_dict,
            emb_seq=emb_seq,
            clus_label_dict=clus_label_dict,
            soft_label_thres=soft_label_thres,
            use_single_scale_clus=use_single_scale_clus,
            window_stride=window_stride,
            seq_eval_mode=seq_eval_mode,
            pairwise_infer=pairwise_infer,
        )

    def msdd_infer_collate_fn(self, batch):
        """Collate batch of audio features, feature lengths, target label sequences for inference."""
        return _msdd_infer_collate_fn(self, batch)


class _AudioToSpeechE2ESpkDiarDataset(Dataset):
    """
    Dataset class that loads a json file containing paths to audio files,
    RTTM files and number of speakers. This Dataset class is designed for
    training or fine-tuning speaker embedding extractor and diarization decoder
    at the same time.

    Example:
    {"audio_filepath": "/path/to/audio_0.wav", "num_speakers": 2,
    "rttm_filepath": "/path/to/diar_label_0.rttm}
    ...
    {"audio_filepath": "/path/to/audio_n.wav", "num_speakers": 2,
    "rttm_filepath": "/path/to/diar_label_n.rttm}

    Args:
        manifest_filepath (str):
            Path to input manifest json files.
        multiargs_dict (dict):
            Dictionary containing the parameters for multiscale segmentation and clustering.
        soft_label_thres (float):
            Threshold that determines the label of each segment based on RTTM file information.
        featurizer:
            Featurizer instance for generating audio_signal from the raw waveform.
        window_stride (float):
            Window stride for acoustic feature. This value is used for calculating the numbers of feature-level frames.
    """

    @property
    def output_types(self) -> Optional[Dict[str, NeuralType]]:
        """Returns definitions of module output ports."""
        output_types = {
            "audio_signal": NeuralType(('B', 'T'), AudioSignal()),
            "audio_length": NeuralType(('B'), LengthsType()),
            "targets": NeuralType(('B', 'T', 'C'), ProbsType()),
            "target_len": NeuralType(('B'), LengthsType()),
        }

        return output_types

    def __init__(
        self,
        *,
        manifest_filepath: str,
        soft_label_thres: float,
        session_len_sec: float,
        num_spks: int,
        featurizer,
        window_stride: float,
        min_subsegment_duration: float = 0.03,
        global_rank: int = 0,
        dtype=torch.float16,
        round_digits: int = 2,
        soft_targets: bool = False,
        subsampling_factor: int = 8,
    ):
        super().__init__()
        self.collection = EndtoEndDiarizationSpeechLabel(
            manifests_files=manifest_filepath.split(','),
            round_digits=round_digits,
        )
        self.featurizer = featurizer
        self.round_digits = round_digits
        self.feat_per_sec = int(1 / window_stride)
        self.diar_frame_length = round(subsampling_factor * window_stride, round_digits)
        self.session_len_sec = session_len_sec
        self.soft_label_thres = soft_label_thres
        self.max_spks = num_spks
        self.min_subsegment_duration = min_subsegment_duration
        self.dtype = dtype
        self.use_asr_style_frame_count = True
        self.soft_targets = soft_targets
        self.round_digits = 2
        self.floor_decimal = 10**self.round_digits

    def __len__(self):
        return len(self.collection)

    def get_uniq_id_with_range(self, sample, deci=3):
        """
        Generate unique training sample ID from unique file ID, offset and duration. The start-end time added
        unique ID is required for identifying the sample since multiple short audio samples are generated from a single
        audio file. The start time and end time of the audio stream uses millisecond units if `deci=3`.

        Args:
            sample:
                `DiarizationSpeechLabel` instance from collections.

        Returns:
            uniq_id (str):
                Unique sample ID which includes start and end time of the audio stream.
                Example: abc1001_3122_6458
        """
        bare_uniq_id = os.path.splitext(os.path.basename(sample.rttm_file))[0]
        offset = str(int(round(sample.offset, deci) * pow(10, deci)))
        endtime = str(int(round(sample.offset + sample.duration, deci) * pow(10, deci)))
        uniq_id = f"{bare_uniq_id}_{offset}_{endtime}"
        return uniq_id

    def parse_rttm_for_targets_and_lens(self, rttm_file, offset, duration, target_len):
        """
        Generate target tensor variable by extracting groundtruth diarization labels from an RTTM file.
        This function converts (start, end, speaker_id) format into base-scale (the finest scale) segment level
        diarization label in a matrix form.

        Example of seg_target:
            [[0., 1.], [0., 1.], [1., 1.], [1., 0.], [1., 0.], ..., [0., 1.]]
        """
        with open(rttm_file, 'r') as f:
            rttm_lines = f.readlines()

        rttm_timestamps, sess_to_global_spkids = extract_frame_info_from_rttm(offset, duration, rttm_lines)

        fr_level_target = get_frame_targets_from_rttm(
            rttm_timestamps=rttm_timestamps,
            offset=offset,
            duration=duration,
            round_digits=self.round_digits,
            feat_per_sec=self.feat_per_sec,
            max_spks=self.max_spks,
        )

        soft_target_seg = self.get_soft_targets_seg(feat_level_target=fr_level_target, target_len=target_len)
        if self.soft_targets:
            step_target = soft_target_seg
        else:
            step_target = (soft_target_seg >= self.soft_label_thres).float()
        return step_target

    def get_soft_targets_seg(self, feat_level_target, target_len):
        """
        Generate the final targets for the actual diarization step.
        Here, frame level means step level which is also referred to as segments.
        We follow the original paper and refer to the step level as "frames".

        Args:
            feat_level_target (torch.tensor):
                Tensor variable containing hard-labels of speaker activity in each feature-level segment.
            target_len (torch.tensor):
                Numbers of ms segments

        Returns:
            soft_target_seg (torch.tensor):
                Tensor variable containing soft-labels of speaker activity in each step-level segment.
        """
        num_seg = torch.max(target_len)
        targets = torch.zeros(num_seg, self.max_spks)
        stride = int(self.feat_per_sec * self.diar_frame_length)
        for index in range(num_seg):
            if index == 0:
                seg_stt_feat = 0
            else:
                seg_stt_feat = stride * index - 1 - int(stride / 2)
            if index == num_seg - 1:
                seg_end_feat = feat_level_target.shape[0]
            else:
                seg_end_feat = stride * index - 1 + int(stride / 2)
            targets[index] = torch.mean(feat_level_target[seg_stt_feat : seg_end_feat + 1, :], axis=0)
        return targets

    def get_segment_timestamps(
        self,
        duration: float,
        offset: float = 0,
        sample_rate: int = 16000,
    ):
        """
        Get start and end time of segments in each scale.

        Args:
            sample:
                `DiarizationSpeechLabel` instance from preprocessing.collections
        Returns:
            segment_timestamps (torch.tensor):
                Tensor containing Multiscale segment timestamps.
            target_len (torch.tensor):
                Number of segments for each scale. This information is used for reshaping embedding batch
                during forward propagation.
        """
        subsegments = get_subsegments(
            offset=offset,
            window=round(self.diar_frame_length * 2, self.round_digits),
            shift=self.diar_frame_length,
            duration=duration,
            min_subsegment_duration=self.min_subsegment_duration,
            use_asr_style_frame_count=self.use_asr_style_frame_count,
            sample_rate=sample_rate,
            feat_per_sec=self.feat_per_sec,
        )
        if self.use_asr_style_frame_count:
            effective_dur = (
                np.ceil((1 + duration * sample_rate) / int(sample_rate / self.feat_per_sec)).astype(int)
                / self.feat_per_sec
            )
        else:
            effective_dur = duration
        ts_tensor = get_subsegments_to_timestamps(
            subsegments, self.feat_per_sec, decimals=2, max_end_ts=(offset + effective_dur)
        )
        target_len = torch.tensor([ts_tensor.shape[0]])
        return target_len

    def __getitem__(self, index):
        sample = self.collection[index]
        if sample.offset is None:
            sample.offset = 0
        offset = sample.offset
        if self.session_len_sec < 0:
            session_len_sec = sample.duration
        else:
            session_len_sec = min(sample.duration, self.session_len_sec)

        audio_signal = self.featurizer.process(sample.audio_file, offset=offset, duration=session_len_sec)

        # We should resolve the length mis-match from the round-off errors between these two variables:
        # `session_len_sec` and `audio_signal.shape[0]`
        session_len_sec = (
            np.floor(audio_signal.shape[0] / self.featurizer.sample_rate * self.floor_decimal) / self.floor_decimal
        )
        audio_signal = audio_signal[: round(self.featurizer.sample_rate * session_len_sec)]

        audio_signal_length = torch.tensor(audio_signal.shape[0]).long()
        audio_signal, audio_signal_length = audio_signal.to('cpu'), audio_signal_length.to('cpu')
        target_len = self.get_segment_timestamps(duration=session_len_sec, sample_rate=self.featurizer.sample_rate)
        targets = self.parse_rttm_for_targets_and_lens(
            rttm_file=sample.rttm_file, offset=offset, duration=session_len_sec, target_len=target_len
        )
        return audio_signal, audio_signal_length, targets, target_len


def _eesd_train_collate_fn(self, batch):
    """
    Collate a batch of variables needed for training the end-to-end speaker diarization (EESD) model
    from raw waveforms to diarization labels. The following variables are included in the training/validation batch:

    Args:
        batch (tuple):
            A tuple containing the variables for diarization training.

    Returns:
        audio_signal (torch.Tensor):
            A tensor containing the raw waveform samples (time series) loaded from the `audio_filepath`
            in the input manifest file.
        feature_length (torch.Tensor):
            A tensor containing the lengths of the raw waveform samples.
        targets (torch.Tensor):
            Groundtruth speaker labels for the given input embedding sequence.
        target_lens (torch.Tensor):
            A tensor containing the number of segments for each sample in the batch, necessary for
            reshaping inputs to the EESD model.
    """
    packed_batch = list(zip(*batch))
    audio_signal, feature_length, targets, target_len = packed_batch
    audio_signal_list, feature_length_list = [], []
    target_len_list, targets_list = [], []

    max_raw_feat_len = max([x.shape[0] for x in audio_signal])
    max_target_len = max([x.shape[0] for x in targets])
    if max([len(feat.shape) for feat in audio_signal]) > 1:
        max_ch = max([feat.shape[1] for feat in audio_signal])
    else:
        max_ch = 1
    for feat, feat_len, tgt, segment_ct in batch:
        seq_len = tgt.shape[0]
        if len(feat.shape) > 1:
            pad_feat = (0, 0, 0, max_raw_feat_len - feat.shape[0])
        else:
            pad_feat = (0, max_raw_feat_len - feat.shape[0])
        if feat.shape[0] < feat_len:
            feat_len_pad = feat_len - feat.shape[0]
            feat = torch.nn.functional.pad(feat, (0, feat_len_pad))
        pad_tgt = (0, 0, 0, max_target_len - seq_len)
        padded_feat = torch.nn.functional.pad(feat, pad_feat)
        padded_tgt = torch.nn.functional.pad(tgt, pad_tgt)
        if max_ch > 1 and padded_feat.shape[1] < max_ch:
            feat_ch_pad = max_ch - padded_feat.shape[1]
            padded_feat = torch.nn.functional.pad(padded_feat, (0, feat_ch_pad))
        audio_signal_list.append(padded_feat)
        feature_length_list.append(feat_len.clone().detach())
        target_len_list.append(segment_ct.clone().detach())
        targets_list.append(padded_tgt)
        audio_signal = torch.stack(audio_signal_list)
    feature_length = torch.stack(feature_length_list)
    target_lens = torch.stack(target_len_list).squeeze(1)
    targets = torch.stack(targets_list)
    return audio_signal, feature_length, targets, target_lens


class AudioToSpeechE2ESpkDiarDataset(_AudioToSpeechE2ESpkDiarDataset):
    """
    Dataset class for loading a JSON file containing paths to audio files,
    RTTM (Rich Transcription Time Marked) files, and the number of speakers.
    This class is designed for training or fine-tuning a speaker embedding
    extractor and diarization decoder simultaneously.

    The JSON manifest file should have entries in the following format:

    Example:
    {
        "audio_filepath": "/path/to/audio_0.wav",
        "num_speakers": 2,
        "rttm_filepath": "/path/to/diar_label_0.rttm"
    }
    ...
    {
        "audio_filepath": "/path/to/audio_n.wav",
        "num_speakers": 2,
        "rttm_filepath": "/path/to/diar_label_n.rttm"
    }

    Args:
        manifest_filepath (str):
            Path to the input manifest JSON file containing paths to audio and RTTM files.
        soft_label_thres (float):
            Threshold for assigning soft labels to segments based on RTTM file information.
        session_len_sec (float):
            Duration of each session (in seconds) for training or fine-tuning.
        num_spks (int):
            Number of speakers in the audio files.
        featurizer:
            Instance of a featurizer for generating features from the raw waveform.
        window_stride (float):
            Window stride (in seconds) for extracting acoustic features, used to calculate
            the number of feature frames.
        global_rank (int):
            Global rank of the current process (used for distributed training).
        soft_targets (bool):
            Whether or not to use soft targets during training.

    Methods:
        eesd_train_collate_fn(batch):
            Collates a batch of data for end-to-end speaker diarization training.
    """

    def __init__(
        self,
        *,
        manifest_filepath: str,
        soft_label_thres: float,
        session_len_sec: float,
        num_spks: int,
        featurizer,
        window_stride,
        global_rank: int,
        soft_targets: bool,
    ):
        super().__init__(
            manifest_filepath=manifest_filepath,
            soft_label_thres=soft_label_thres,
            session_len_sec=session_len_sec,
            num_spks=num_spks,
            featurizer=featurizer,
            window_stride=window_stride,
            global_rank=global_rank,
            soft_targets=soft_targets,
        )

    def eesd_train_collate_fn(self, batch):
        """Collate a batch of data for end-to-end speaker diarization training."""
        return _eesd_train_collate_fn(self, batch)


#Extraction

In [ ]:
#Extraction des fichiers de l'archive .neno pour le repertoire courant
checkpoint_path = "/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/finetuning_modele_de_diarization/diar_sortformer_4spk-v1.nemo"
!tar -xf /content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/finetuning_modele_de_diarization/diar_sortformer_4spk-v1.nemo


In [ ]:
#vérification des fichiers extraits . pour pouvoir le voir , il faut permettre les fichiers qui sont cachés.
!ls -lh /content/


total 471M
drwx------  6 root root 4.0K May  3 08:44 gdrive
-rwxrwxrwx  1 1001 1001 3.4K Nov 27 06:51 model_config.yaml
-rwxrwxrwx  1 1001 1001 471M Oct 23  2024 model_weights.ckpt
drwxr-xr-x 13 root root 4.0K May  3 08:32 NeMo
drwxr-xr-x  1 root root 4.0K Apr 30 13:37 sample_data
drwxr-xr-x  3 root root 4.0K May  3 08:59 sortformer_diar_train


#Copie des fichiers d'un point A à un point B

In [ ]:
#copie des fichiers de configuration et poid dans le drive
#!cp -r /content/model_config.yaml /content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/finetuning_modele_de_diarization/
!cp -r /content/model_weights.ckpt /content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/finetuning_modele_de_diarization/


In [ ]:
import torch

checkpoint_path = "/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/finetuning_modele_de_diarization/model_weights.ckpt"
checkpoint = torch.load(checkpoint_path, map_location="cuda")
print(checkpoint.keys())  # Check if 'pytorch-lightning_version' is present


odict_keys(['preprocessor.featurizer.window', 'preprocessor.featurizer.fb', 'encoder.pre_encode.out.weight', 'encoder.pre_encode.out.bias', 'encoder.pre_encode.conv.0.weight', 'encoder.pre_encode.conv.0.bias', 'encoder.pre_encode.conv.2.weight', 'encoder.pre_encode.conv.2.bias', 'encoder.pre_encode.conv.3.weight', 'encoder.pre_encode.conv.3.bias', 'encoder.pre_encode.conv.5.weight', 'encoder.pre_encode.conv.5.bias', 'encoder.pre_encode.conv.6.weight', 'encoder.pre_encode.conv.6.bias', 'encoder.layers.0.norm_feed_forward1.weight', 'encoder.layers.0.norm_feed_forward1.bias', 'encoder.layers.0.feed_forward1.linear1.weight', 'encoder.layers.0.feed_forward1.linear1.bias', 'encoder.layers.0.feed_forward1.linear2.weight', 'encoder.layers.0.feed_forward1.linear2.bias', 'encoder.layers.0.norm_conv.weight', 'encoder.layers.0.norm_conv.bias', 'encoder.layers.0.conv.pointwise_conv1.weight', 'encoder.layers.0.conv.pointwise_conv1.bias', 'encoder.layers.0.conv.depthwise_conv.weight', 'encoder.layers